In [1]:
%load_ext autoreload
%autoreload 2

In [66]:
from pnn.layer.input import InputLayer
from pnn.tensor import Tensor
from pnn.layer.fully_connected import FullyConnected
from pnn.layer.activation import ActivationLayer, sigmoid, soft_max, relu
from pnn.layer.loss import LossLayer, mean_squared_error, cross_entropy
from pnn.network import Network
from pnn.trainer import Trainer, sgd
from pnn.shape import Shape
import mnist
import numpy as np 

In [72]:
num_filters=2
kernel_size=Shape((2, 2))
weights = Tensor(
            elements=np.array(
                [[[ [ 0.1, 0.37],
                    [ 0.7, 0.9 ]],

                    [[ 0.3, 0.32],
                    [ 0.9, 0.2 ]]],


                    [[[-0.2, -0.9 ],
                    [ 0.6, 0.3 ]],

                    [[ 0.4, 0.17],
                    [-1.1, -0.7 ]]]]
            )
        )
out_tensor = Tensor(
            elements=np.array(
                [   [[ 2.,      1.469 ],
                    [ 2.123,  -0.1288]],

                    [[-0.34,   -0.784 ],
                    [-3.83,   -3.689 ]],

                    [[-0.83,   -1.464 ],
                    [ 2.06,   -1.984 ]]],
                dtype=np.float64,
            )
        )

out_tensor.deltas = np.array(
            [[  [ 0.1,  -0.5 ],
                [-0.25, -0.8 ]],

                [[ 0.33,  0.2 ],
                [ 1.3,   0.81]],

                [[-0.6,   0.1 ],
                [ 0.01,  1.1 ]]],
            dtype=np.float64,
        )
in_tensor = Tensor(
            elements=np.array(
                [[  [ 0.1, 0.9 ],
                    [ 1.2, 1.1 ],
                    [ 0.01, 3.2 ]],

                    [[-0.2, 0.3 ],
                    [ 1.4, 0.7 ],
                    [ 0.2, 1.7 ]],

                    [[ 0.5, 0.5 ],
                    [ 1.6, 2.2 ],
                    [-0.3, 6.3 ]],

                    [[ 0.6, 0.65],
                    [ 2.2, 4.4 ],
                    [ 4., 8.2 ]]]
            )
        )

In [95]:
for i in range(num_filters):
    for m in range(in_shape.shape[2]):
        for j in range(kernel_size.shape[0]):
            for k in range(kernel_size.shape[1]):
            
                print(weights.elements[j][k][m][i])

0.1
0.3
-0.2
0.4
0.7
0.9
0.6
-1.1
0.37
0.32
-0.9
0.17
0.9
0.2
0.3
-0.7


In [73]:
print(0.1*0.1 + 0.3*1.2 + (-0.2)*(-0.2) + 0.4*1.4)
print(0.7*0.9 + 0.9*1.1 +  0.6*(0.3) + -1.1*0.7)

0.97
1.03


In [149]:
out_tensors = [Tensor(elements=np.zeros((3, 2, 2), dtype=np.float64))]

In [151]:
output_arr = []
for y in range(len(tensor.elements) - kernel_size.shape[0] + 1):
    y_arr = []
    for x in range(len(tensor.elements[0]) - kernel_size.shape[1] + 1):
        x_arr = []
        for z in range(num_filters):
            #for a in range(in_shape.shape[2]):
                #for i in range(kernel_size.shape[0]):
                #    for j in range(kernel_size.shape[1]):
                        #print(tensor.elements[y + i][x + j][a])
                        #print(weights.elements[i][j][a][z])
                        #c.append(np.multiply(tensor.elements[y + i][x + j][a], weights.elements[i][j][a][z]))
            channel_value = np.sum([np.multiply(tensor.elements[y + i][x + j][a], weights.elements[i][j][a][z]) for j in range(kernel_size.shape[1]) for i in range(kernel_size.shape[0]) for a in range(in_shape.shape[2])])
                #channel_values = np.sum(channel_values, axis=0)
            x_arr.append(channel_value)
        y_arr.append(x_arr)
    output_arr.append(y_arr)
out_tensors[0].elements = np.array(output_arr)

In [153]:
out_tensors[0].elements

array([[[ 2.    ,  1.469 ],
        [ 2.123 , -0.1288]],

       [[-0.34  , -0.784 ],
        [-3.83  , -3.689 ]],

       [[-0.83  , -1.464 ],
        [ 2.06  , -1.984 ]]])

In [134]:
np.sum(c[8:16])

1.469

In [24]:
arr1 = np.array([0.1, 0.9])
arr2 = np.array([2, 2])

In [25]:
np.multiply(arr1, arr2)

array([0.2, 1.8])

In [19]:
np.sum(arr1, axis=0)

array([2.1, 2.9])

In [35]:
weights.elements[0][1][1]

array([0.9, 0.2])

In [100]:
c

[array([ 0.01, -0.06, -0.24,  0.56]),
 array([ 0.07, -0.18,  0.72, -1.54]),
 array([ 0.333,  0.096, -0.99 ,  0.119]),
 array([ 0.81,  0.06,  0.33, -0.49]),
 array([ 0.12 ,  0.42 , -0.002,  0.08 ]),
 array([ 0.84 ,  1.26 ,  0.006, -0.22 ]),
 array([ 0.407,  0.224, -2.88 ,  0.289]),
 array([ 0.99,  0.14,  0.96, -1.19]),
 array([-0.02,  0.15, -0.28,  0.64]),
 array([-0.14,  0.45,  0.84, -1.76]),
 array([ 0.111,  0.16 , -0.63 ,  0.374]),
 array([ 0.27,  0.1 ,  0.21, -1.54]),
 array([ 0.14,  0.48, -0.04, -0.12]),
 array([0.98, 1.44, 0.12, 0.33]),
 array([ 0.259,  0.704, -1.53 ,  1.071]),
 array([ 0.63,  0.44,  0.51, -4.41]),
 array([ 0.05,  0.18, -0.32,  0.88]),
 array([ 0.35,  0.54,  0.96, -2.42]),
 array([ 0.185,  0.208, -1.98 ,  0.748]),
 array([ 0.45,  0.13,  0.66, -3.08]),
 array([0.16, 0.66, 0.06, 1.6 ]),
 array([ 1.12,  1.98, -0.18, -4.4 ]),
 array([ 0.814,  1.408, -5.67 ,  1.394]),
 array([ 1.98,  0.88,  1.89, -5.74])]

# Items

In [67]:
# mnist.temporary_dir = lambda: './data'
train_images = mnist.train_images()
train_images = np.divide(train_images, np.max(train_images))
train_labels = mnist.train_labels()

test_images = mnist.test_images()
test_images = np.divide(test_images, np.max(test_images))
test_labels = mnist.test_labels()

# train_images = train_images[5:5000]
# train_labels = train_labels[5:5000]
# train_labels = np.array(train_labels)

# Trainer

In [70]:
# fully_connected_1 = FullyConnected(out_shape=Shape((784,)), initialization_technique='sigmoid')
# activation_layer_sigmoid1 = ActivationLayer(sigmoid)
fully_connected_2 = FullyConnected(out_shape=Shape((196,)), initialization_technique='sigmoid')
activation_layer_sigmoid2 = ActivationLayer(sigmoid)
# fully_connected_3 = FullyConnected(out_shape=Shape((49,)), initialization_technique='sigmoid')
# activation_layer_sigmoid3 = ActivationLayer(sigmoid)
fully_connected_4 = FullyConnected(out_shape=Shape((10,)), initialization_technique='softmax') 
activation_layer_soft_max = ActivationLayer(soft_max)
loss_layer = LossLayer(cross_entropy)
layerlist = [#fully_connected_1, activation_layer_sigmoid1, 
             fully_connected_2, activation_layer_sigmoid2,
            #  fully_connected_3, activation_layer_sigmoid3,
             fully_connected_4, activation_layer_soft_max, loss_layer]
network = Network(layerlist)

sgd_trainer = Trainer(
    learning_rate=0.03,
    amount_epochs=20,
    update_mechanism=sgd,
    batch_size=1)

In [71]:
sgd_trainer.optimize(network=network, data=train_images, labels=train_labels)

epoche: 0, loss: 2.3082, time: 73.5879s


KeyboardInterrupt: 

# Accuracy

In [ ]:
prediction = network.predict(test_images)
print(sum(prediction == test_labels)/len(test_labels))

0.9547


# Save and load

In [8]:
network.save_network('./networks/neural_network_2_layer')

In [32]:
n = Network.load_network('./networks/neural_network')

In [30]:
prediction = n.predict(test_images)
print(sum(prediction == test_labels)/len(test_labels))

0.9519


# Network

In [67]:
fully_connected_1 = FullyConnected(out_shape=Shape((15,)), initialization_technique='sigmoid')
activation_layer_sigmoid = ActivationLayer(sigmoid)
fully_connected_2 = FullyConnected(out_shape=Shape((10,)), initialization_technique='softmax') 
activation_layer_soft_max = ActivationLayer(soft_max)
loss_layer = LossLayer(mean_squared_error)
layerlist = [fully_connected_1, activation_layer_sigmoid, fully_connected_2, activation_layer_soft_max, loss_layer]
network = Network(layerlist)

In [43]:
train_labels

array([5, 0, 4, ..., 5, 6, 8], dtype=uint8)

In [87]:
output = network.forward(train_images, train_labels,unique_length=10)
output

0.11209011367433079

In [70]:
network.tensorlist[2][0].elements

array([1.00000000e+00, 1.00000000e+00, 1.00000000e+00, 1.33377769e-74,
       4.14081228e-05, 1.00000000e+00, 1.00000000e+00, 1.00000000e+00,
       1.00000000e+00, 1.00000000e+00, 3.45260849e-25, 1.00000000e+00,
       1.00000000e+00, 9.99973057e-01, 9.99999372e-01])

In [71]:
network.backprop()

In [74]:
network.tensorlist[1][0].elements

array([  27.69562386,  171.75035953,  313.72672599, -170.10328159,
        -10.09199209,   87.46834609,   94.59948139,  154.94512526,
        181.95918689,  251.6944964 ,  -56.3254973 ,  199.59030414,
        181.73033401,   10.52174898,   14.28127379])

In [86]:
for i in network.layers[0].weights.deltas:
    if np.max(i) > 0:
        print(i)

[ 1.65874242e-14  0.00000000e+00  0.00000000e+00 -4.16890222e-77
  3.36041250e-07  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  1.61056219e-27  0.00000000e+00
  0.00000000e+00  8.18070301e-07  3.02447427e-09]
[ 9.95245452e-14  0.00000000e+00  0.00000000e+00 -2.50134133e-76
  2.01624750e-06  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  9.66337314e-27  0.00000000e+00
  0.00000000e+00  4.90842181e-06  1.81468456e-08]
[ 9.95245452e-14  0.00000000e+00  0.00000000e+00 -2.50134133e-76
  2.01624750e-06  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  9.66337314e-27  0.00000000e+00
  0.00000000e+00  4.90842181e-06  1.81468456e-08]
[ 9.95245452e-14  0.00000000e+00  0.00000000e+00 -2.50134133e-76
  2.01624750e-06  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  9.66337314e-27  0.00000000e+00
  0.00000000e+00  4.90842181e-06  1.81468456e-08]
[ 6.96671817e-13  0.

# Input Layer

In [ ]:
input_layer = InputLayer()
input_tensorlist = input_layer.forward(train_images)

# Fully Connected 1

In [ ]:
out_shape=(100,)
fully_connected = FullyConnected(out_shape=out_shape)
out_tensor_fully_connected1 = np.array([Tensor(np.random.rand(out_shape[0]), None) for j in range(0, len(input_tensorlist))])
fully_connected.forward(in_tensors=input_tensorlist, out_tensors=out_tensor_fully_connected1)

In [ ]:
out_tensor_fully_connected1[0].elements

array([13855.32327516, 14567.87946531, 12531.64266387, 13065.09199568,
       13717.27042773, 13144.85267195, 14172.86492735, 13344.64554154,
       13789.30046119, 13234.91262216, 14386.86711745, 14110.44445943,
       14011.03393467, 13626.79211007, 14183.92327854, 13196.15787959,
       12533.8216709 , 14009.36439561, 14305.71061599, 14244.94249371,
       13756.82421686, 14248.35678187, 13694.60728455, 13465.37950326,
       13765.26222772, 14410.22192256, 13453.50571786, 13971.10785787,
       13484.35685788, 12655.63794863, 12429.55267367, 14555.04290196,
       13832.72866545, 14343.43362097, 13755.93510764, 13823.52715719,
       13342.75422834, 12687.45679859, 13605.87063681, 13682.76244315,
       13272.07845833, 12832.37884452, 14254.27618778, 12977.85169336,
       15051.38516363, 14307.03311969, 14245.05595451, 12788.95005566,
       13792.19609532, 13385.96231409, 13881.2569936 , 13664.41136983,
       14123.34368646, 13645.73949912, 14719.56784325, 13915.15399392,
      

# Sigmoid

In [ ]:
activation_layer = ActivationLayer(sigmoid)
out_tensor_activation = np.array([Tensor(np.random.rand(out_shape[0]), None) for j in range(0, len(input_tensorlist))])
activation_layer.forward(out_tensor_fully_connected1, out_tensor_activation)

In [ ]:
sigmoid_layer = ActivationLayer(sigmoid)

In [ ]:
in_tensors = [Tensor(elements=np.array([1, 2, 3, 4], dtype=np.float64))]
out_tensors = [Tensor(elements=np.array([0, 0, 0, 0], dtype=np.float64))]
out_tensors[0].deltas = np.array([6, 7, 8, 9])
expected_output = [1.1796716, 0.7349551, 0.36141328, 0.15896436]

sigmoid_layer.forward(in_tensors, out_tensors)
sigmoid_layer.backward(out_tensors, in_tensors)

In [ ]:
in_tensors[0].deltas

array([1.1796716 , 0.7349551 , 0.36141328, 0.15896436])

# Fully Connected 2

In [ ]:
out_shape=(10,)
fully_connected = FullyConnected(out_shape=out_shape)
out_tensor_fully_connected2 = np.array([Tensor(np.random.rand(out_shape[0]), None) for j in range(0, len(input_tensorlist))])
fully_connected.forward(in_tensors=out_tensor_activation, out_tensors=out_tensor_fully_connected2)

In [ ]:
out_tensor_fully_connected2[0].elements

array([21.53422881, 23.77821393, 21.3411817 , 23.2444338 , 22.3820653 ,
       20.77149952, 22.49382576, 19.6519166 , 23.03132047, 19.78697478])

# Soft Max

In [ ]:
activation_layer = ActivationLayer(soft_max)
out_tensor_activation = np.array([Tensor(np.random.rand(out_shape[0]), None) for j in range(0, len(input_tensorlist))])
activation_layer.forward(out_tensor_fully_connected2, out_tensor_activation)

In [ ]:
print(len(out_tensor_activation[0].elements))
out_tensor_activation[0].elements

10


array([0.03704778, 0.34939171, 0.03054379, 0.20487777, 0.08649141,
       0.01727878, 0.09671858, 0.00564007, 0.16555446, 0.00645564])

# Mean Squared Error

In [ ]:
mse_layer = LossLayer(mean_squared_error)
cross_entropy_layer = LossLayer(loss=cross_entropy)
pred = [Tensor(elements=np.array([0.1, 0.2, 0.3, 0.4], dtype=np.float64))]
target = [Tensor(elements=np.array([1, 0, 0, 0], dtype=np.float64))]
mse = mse_layer.forward(targets=target, predictions=pred)
cross_entropy_val = cross_entropy_layer.forward(targets=target, predictions=pred)

In [ ]:
cross_entropy_val

2.3025850929940455